<a href="https://colab.research.google.com/github/andrei-radulescu-banu/stat453-deep-learning-ss21/blob/main/L15/1_lstm_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U torch==1.13.0 torchtext==0.14.0 torchdata==0.5.0

# Reload environment
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 12.4 MB/s 
     |████████████████████████████████| 140 kB 58.5 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


STAT 453: Deep Learning (Spring 2021)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)  

Course website: http://pages.stat.wisc.edu/~sraschka/teaching/stat453-ss2021/  
GitHub repository: https://github.com/rasbt/stat453-deep-learning-ss21

---

In [1]:
!pip install watermark
!pip install colab-env --upgrade
import colab_env
colab_env.envvar_handler.add_env("CUBLAS_WORKSPACE_CONFIG", ":4096:8", overwrite=True)
!git clone https://github.com/andrei-radulescu-banu/stat453-deep-learning-ss21.git
import sys, os
sys.path.append("/content/stat453-deep-learning-ss21/L15")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3837 sha256=58c313bb6f37db82ec91eb82fe8b7a4f8446e037917ccb9de09c583ef4c19aaa
  Stored in directory: /root/.cache/pip/wheels/1c/65/0c/5552431f2622d6e0283e3dba61c6837103a9cbdbd89b7b0cba
Successfully built colab-env
Mounted at /content/gdrive
Cloning into 'stat453-deep-learning-ss21'...
remote: Enumerating objects: 1311, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 1311 (delta 179), reused 179 (delta 135), pack-reused 1050
Receiving objects: 100% (1311/1311), 116.80 MiB | 25.96 MiB/s, done.
Resolving deltas: 100% (283/283), done.


# RNN Classifier with LSTM Trained on Own Dataset (IMDB)

Example notebook showing how to use an own CSV text dataset for training a simple RNN for sentiment classification (here: a binary classification problem with two labels, positive and negative) using LSTM (Long Short Term Memory) cells.

In [2]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch,torchtext

import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd

torch.backends.cudnn.deterministic = True

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.8.15
IPython version      : 7.9.0

torch    : 1.13.0+cu116
torchtext: 0.14.0



## General Settings

In [3]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

## Download Dataset

The following cells will download the IMDB movie review dataset (http://ai.stanford.edu/~amaas/data/sentiment/) for positive-negative sentiment classification in as CSV-formatted file:

In [4]:
!wget https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz

--2022-12-08 01:36:18--  https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz [following]
--2022-12-08 01:36:18--  https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘movie_data.csv.gz’

movie_data.csv.gz   100%[===================>]  25.29M  --.-KB/s    in 0.08s   

2

In [5]:
!gunzip -f movie_data.csv.gz 

Check that the dataset looks okay:

In [6]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [7]:
del df

## Prepare Dataset with Torchdata and the new DataPipes API

In [8]:
from torchdata.datapipes.iter import IterableWrapper, FileOpener
datapipe = IterableWrapper(["movie_data.csv"])
datapipe = FileOpener(datapipe, mode='b')
datapipe = datapipe.parse_csv(skip_lines=1)

for sample in datapipe:
     print(sample)
     break

['In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich fami

In [57]:
# Get the number of rows in dataset
N_ROWS = len(list(datapipe))  # 50000

# Split into training and val datapipes early on. Will build vocabulary from training datapipe only.
#train_dp, val_dp = datapipe.random_split(total_length=N_ROWS, weights={"train": 0.5, "valid": 0.5}, seed=0)
train_dp, val_dp, test_dp = datapipe.random_split(total_length=N_ROWS, weights={"train": 0.5, "valid": 0.4, "test": 0.1}, seed=0)

In [23]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

# See tutorial at https://www.youtube.com/watch?v=CrS-LFXEiyk
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)


def get_vocab(train_datapipe):
    vocab = build_vocab_from_iterator(yield_tokens(train_datapipe), specials=["<unk>"], max_tokens=VOCABULARY_SIZE)
    vocab.set_default_index(vocab["<unk>"])
    return vocab

In [24]:
vocab = get_vocab(train_dp)

# List vocabulary strings in order
print(vocab.get_itos()[:5])

from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

# List five vocabulary items
print(take(5, vocab.get_stoi().items()))

['<unk>', 'the', '.', ',', 'and']
[('railsback', 19997), ('raider', 19996), ('radu', 19995), ('r2', 19993), ('quiz', 19992)]


In [26]:
# Pipeline word tokenizer: 
# - convert reviews to word tokens, and word tokens to vocab index
# - convert rating to number in {0, 1}
def tokenize_pipeline(sample) : return (vocab(list(tokenizer(sample[0]))), int(sample[1]))

In [25]:
def build_data_pipe(dp, mode, len=1000, batch_size=BATCH_SIZE):
  # Tokenize stage
  dp = dp.map(tokenize_pipeline)

  if mode == "train":
    dp = dp.shuffle(buffer_size=len)
  
  dp = dp.sharding_filter()
  # important to use sharding_filter after (not before) shuffling

  if mode == "train":
    dp = dp.batch(batch_size=batch_size, drop_last=True)

  elif mode == "validate":
    dp = dp.batch(batch_size=batch_size, drop_last=False)

  else:
    raise ValueError(f"Invalid mode argument {mode}.")

  return dp


In [27]:
# Apply pipeline stage
train_dp1 = build_data_pipe(train_dp, "train")
val_dp1 = build_data_pipe(val_dp, "validate")
for sample in train_dp1:
     print(sample)
     break

[([13, 12554, 12272, 3, 5540, 3, 7276, 2536, 4646, 837, 776, 12836, 18, 5, 6237, 4356, 4, 9040, 0, 14, 8, 15, 5, 2792, 7, 5838, 2, 4470, 10854, 798, 832, 541, 1824, 24, 40, 0, 0, 7416, 7427, 23, 15265, 7, 104, 146, 51, 6, 31, 0, 1831, 125, 2, 1824, 226, 31, 202, 1016, 59, 731, 837, 17727, 14553, 24, 40, 1348, 7273, 1679, 23, 0, 96, 40, 796, 0, 7661, 69, 16122, 3631, 8556, 1819, 7, 40, 178, 753, 2, 1824, 676, 314, 11270, 849, 11, 595, 1111, 59, 29, 2156, 14553, 8, 15, 301, 2, 3031, 0, 4860, 8, 15, 0, 584, 781, 6, 93, 184, 697, 8, 26, 218, 54, 38, 8, 185, 10156, 67, 7, 33, 3, 196, 122, 3, 37, 1611, 272, 3352, 4, 14, 0, 178, 0, 49, 93, 837, 158, 8, 26, 936, 500, 44, 5, 1432, 1162, 7, 5, 386, 4347, 6, 0, 1375, 2498, 1257, 3, 5, 590, 6667, 3597, 134, 8, 15, 8141, 7155, 19, 203, 57, 4, 484, 3, 5, 338, 287, 6, 0, 3256, 3, 1, 19377, 19820, 103, 1, 582, 9, 1259, 3, 4, 1, 3131, 18578, 6, 1, 10592, 0, 1572, 0, 8, 2057, 8, 37, 18719, 109, 11, 40, 2814, 5810, 4, 458, 829, 5339, 14862, 623, 24, 997,

In [51]:
def text_pipeline(x): return vocab(list(tokenizer(x)))
def label_pipeline(x): return int(x)
    
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(DEVICE), label_list.to(DEVICE), offsets.to(DEVICE)

In [52]:
from torch.utils.data import DataLoader
import torchdata.datapipes as dp
from torchtext.data.functional import to_map_style_dataset

train_dataset = to_map_style_dataset(train_dp)
val_dataset = to_map_style_dataset(val_dp)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [56]:
for batch_idx, batch_data in enumerate(train_dataloader):
  print(batch_idx, batch_data)
  break

0 (tensor([ 777,  156,   13,  ...,    1, 1349, 3175], device='cuda:0'), tensor([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 0, 0], device='cuda:0'), tensor([    0,   104,   287,   368,   480,   621,   828,   980,  1714,  1747,
         1884,  2061,  2867,  3400,  3639,  3771,  4142,  4516,  4856,  5180,
         5290,  5439,  5872,  6037,  6612,  6869,  7251,  7320,  7450,  8193,
         8636,  8808,  8952,  9502,  9724,  9861, 10027, 10219, 10399, 10550,
        10979, 11228, 11548, 11609, 11813, 11859, 12427, 12534, 13790, 13935,
        14167, 14406, 15123, 15754, 15982, 16096, 16592, 16726, 16952, 17126,
        1

In [32]:
from torchtext.data.utils import(
    get_tokenizer,
    ngrams_iterator,
)
tokenizer = get_tokenizer("basic_english")

In [ ]:
### Defining the feature processing

TEXT = torchtext.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.data.LabelField(dtype=torch.long)

In [11]:
### Defining the feature processing

TEXT = torchtext.legacy.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)



Process the dataset:


In [12]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path='movie_data.csv', format='csv',
    skip_header=True, fields=fields)

## Split Dataset into Train/Validation/Test

Split the dataset into training, validation, and test partitions:

In [13]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 40000
Num Test: 10000


In [14]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')

Num Train: 34000
Num Validation: 6000


In [15]:
print(vars(train_data.examples[0]))

{'TEXT_COLUMN_NAME': ['For', 'some', 'unknown', 'reason', ',', '7', 'years', 'ago', ',', 'I', 'watched', 'this', 'movie', 'with', 'my', 'mother', 'and', 'sister', '.', 'I', 'do', "n't", 'think', 'I', "'ve", 'ever', 'laughed', 'as', 'hard', 'with', 'them', 'before', '.', 'This', 'movie', 'was', 'sooooo', 'bad', '.', 'How', 'sequels', 'were', 'produced', 'is', 'beyond', 'me', '.', 'Its', 'been', 'awhile', 'since', 'I', 'last', 'saw', 'this', '"', 'movie', '"', ',', 'but', 'the', 'one', 'impression', 'that', 'it', 'has', 'stuck', 'with', 'me', 'over', 'the', 'years', 'has', 'been', ',', '"', 'They', 'must', 'have', 'found', 'the', 'script', 'in', 'a', 'dumpster', 'in', 'the', 'backlot', 'of', 'a', 'cheap', 'movie', 'studio', ',', 'made', 'into', 'a', '"', 'movie', '"', ',', 'and', 'decided', 'that', 'it', 'did', "n't", 'suck', 'enough', ',', 'and', 'made', 'it', 'worse', '.', 'I', "'m", 'pretty', 'sure', 'that', 'they', 'spent', 'all', 'the', 'budget', 'on', 'camera', 'work', 'and', 'the'

## Build Vocabulary

Build the vocabulary based on the top "VOCABULARY_SIZE" words:

In [16]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


- 20,002 not 20,000 because of the `<unk>` and `<pad>` tokens
- PyTorch RNNs can deal with arbitrary lengths due to dynamic graphs, but padding is necessary for padding sequences to the same length in a given minibatch so we can store those in an array

**Look at most common words:**

In [17]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 391539), (',', 370979), ('.', 319271), ('and', 210725), ('a', 210397), ('of', 194700), ('to', 180225), ('is', 145689), ('in', 118828), ('I', 105650), ('it', 103414), ('that', 93700), ('"', 86164), ("'s", 83068), ('this', 81424), ('-', 71703), ('/><br', 68865), ('was', 67523), ('as', 57974), ('with', 57558)]


**Tokens corresponding to the first 10 indices (0, 1, ..., 9):**

In [18]:
print(TEXT.vocab.itos[:10]) # itos = integer-to-string

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


**Converting a string to an integer:**

In [19]:
print(TEXT.vocab.stoi['the']) # stoi = string-to-integer

2


**Class labels:**

In [20]:
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


**Class label count:**

In [21]:
LABEL.vocab.freqs

Counter({'0': 16981, '1': 17019})

## Define Data Loaders

In [22]:
train_loader, valid_loader, test_loader = \
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )

Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

In [23]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([1136, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([55, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([52, 128])
Target vector size: torch.Size([128])


## Model

In [24]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

In [25]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

## Training

In [26]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/266 | Loss: 0.7058


In [ ]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][0].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.9804818034172058

In [ ]:
print('Probability negative:')
1-predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability negative:


0.9933403395116329

In [ ]:
%watermark -iv

pandas   : 1.2.3
spacy    : 3.0.5
torchtext: 0.9.1
torch    : 1.8.1

